# Import 

In [1]:
from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Read

In [29]:
trn = pd.read_csv('../data/input/train_ver2.csv')

/home/ubuntu/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (5,8,11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# 目的変数：新規購買件数

In [30]:
# 製品の変数を別途に保存しておきます。
prods = trn.columns[24:].tolist()

# 商品変数の欠損値をあらかじめ0に代入しておきます。
trn[prods] = trn[prods].fillna(0.0).astype(np.int8)

# 24個の商品を一つも保有していない顧客データを除去します・
no_product = trn[prods].sum(axis=1) == 0
trn = trn[~no_product]

trn = trn[['ncodpers', 'fecha_dato'] + prods]

In [31]:
trn.head(3)

,ncodpers,fecha_dato,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,1375586,2015-01-28,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1050611,2015-01-28,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1050612,2015-01-28,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
trn.dtypes

ncodpers              int64
fecha_dato           object
ind_ahor_fin_ult1      int8
ind_aval_fin_ult1      int8
ind_cco_fin_ult1       int8
ind_cder_fin_ult1      int8
ind_cno_fin_ult1       int8
ind_ctju_fin_ult1      int8
ind_ctma_fin_ult1      int8
ind_ctop_fin_ult1      int8
ind_ctpp_fin_ult1      int8
ind_deco_fin_ult1      int8
ind_deme_fin_ult1      int8
ind_dela_fin_ult1      int8
ind_ecue_fin_ult1      int8
ind_fond_fin_ult1      int8
ind_hip_fin_ult1       int8
ind_plan_fin_ult1      int8
ind_pres_fin_ult1      int8
ind_reca_fin_ult1      int8
ind_tjcr_fin_ult1      int8
ind_valo_fin_ult1      int8
ind_viv_fin_ult1       int8
ind_nomina_ult1        int8
ind_nom_pens_ult1      int8
ind_recibo_ult1        int8
int_date               int8
dtype: object

In [38]:
# 日付を数字に変換する関数です。 2015-01-28は 1, 2016-06-28は 18に変換します。
def date_to_int(str_date):
    Y, M, D = [int(a) for a in str_date.strip().split('-')]
    int_date = (int(Y) - 2015) * 12 + int(M)
    return int_date

# 日付を数字に変換し int_dateに保存します。
trn['int_date'] = trn['fecha_dato'].map(date_to_int).astype(np.int8)

In [41]:
trn.head(3)

,ncodpers,fecha_dato,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,...,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1,int_date
0,1375586,2015-01-28,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1050611,2015-01-28,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1050612,2015-01-28,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [47]:
print(trn.columns)

print(len(trn.columns))

Index(['ncodpers', 'fecha_dato', 'ind_ahor_fin_ult1', 'ind_aval_fin_ult1',
       'ind_cco_fin_ult1', 'ind_cder_fin_ult1', 'ind_cno_fin_ult1',
       'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1',
       'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1', 'ind_deme_fin_ult1',
       'ind_dela_fin_ult1', 'ind_ecue_fin_ult1', 'ind_fond_fin_ult1',
       'ind_hip_fin_ult1', 'ind_plan_fin_ult1', 'ind_pres_fin_ult1',
       'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1',
       'ind_viv_fin_ult1', 'ind_nomina_ult1', 'ind_nom_pens_ult1',
       'ind_recibo_ult1', 'int_date'],
      dtype='object')
27


In [39]:
# データをコピーし, int_date 日付に1を加え lagを生成します。変数名に _prevを追加します。
trn_lag = trn.copy()
trn_lag.columns = [col + '_prev' if col not in ['ncodpers', 'int_date'] else col for col in trn.columns ]
trn_lag['int_date'] += 1

In [40]:
trn_lag.head(3)

,ncodpers,fecha_dato_prev,ind_ahor_fin_ult1_prev,ind_aval_fin_ult1_prev,ind_cco_fin_ult1_prev,ind_cder_fin_ult1_prev,ind_cno_fin_ult1_prev,ind_ctju_fin_ult1_prev,ind_ctma_fin_ult1_prev,ind_ctop_fin_ult1_prev,...,ind_plan_fin_ult1_prev,ind_pres_fin_ult1_prev,ind_reca_fin_ult1_prev,ind_tjcr_fin_ult1_prev,ind_valo_fin_ult1_prev,ind_viv_fin_ult1_prev,ind_nomina_ult1_prev,ind_nom_pens_ult1_prev,ind_recibo_ult1_prev,int_date
0,1375586,2015-01-28,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,1050611,2015-01-28,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,1050612,2015-01-28,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [48]:
print(trn_lag.columns)

print(len(trn_lag.columns))

Index(['ncodpers', 'fecha_dato_prev', 'ind_ahor_fin_ult1_prev',
       'ind_aval_fin_ult1_prev', 'ind_cco_fin_ult1_prev',
       'ind_cder_fin_ult1_prev', 'ind_cno_fin_ult1_prev',
       'ind_ctju_fin_ult1_prev', 'ind_ctma_fin_ult1_prev',
       'ind_ctop_fin_ult1_prev', 'ind_ctpp_fin_ult1_prev',
       'ind_deco_fin_ult1_prev', 'ind_deme_fin_ult1_prev',
       'ind_dela_fin_ult1_prev', 'ind_ecue_fin_ult1_prev',
       'ind_fond_fin_ult1_prev', 'ind_hip_fin_ult1_prev',
       'ind_plan_fin_ult1_prev', 'ind_pres_fin_ult1_prev',
       'ind_reca_fin_ult1_prev', 'ind_tjcr_fin_ult1_prev',
       'ind_valo_fin_ult1_prev', 'ind_viv_fin_ult1_prev',
       'ind_nomina_ult1_prev', 'ind_nom_pens_ult1_prev',
       'ind_recibo_ult1_prev', 'int_date'],
      dtype='object')
27


In [50]:
# 原本データと lag データを ncodperと int_date を基準として合わせます。lag データの int_dateは 1 だけ押されているため、前の月の製品情報が挿入されます。
df_trn = trn.merge(trn_lag, on=['ncodpers','int_date'], how='left')

# メモリの効率化のために、不必要な変数をメモリから除去します。
del trn, trn_lag

In [60]:
print(df_trn.shape)
df_trn.head(3)

(11091070, 52)


,ncodpers,fecha_dato,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,...,ind_hip_fin_ult1_prev,ind_plan_fin_ult1_prev,ind_pres_fin_ult1_prev,ind_reca_fin_ult1_prev,ind_tjcr_fin_ult1_prev,ind_valo_fin_ult1_prev,ind_viv_fin_ult1_prev,ind_nomina_ult1_prev,ind_nom_pens_ult1_prev,ind_recibo_ult1_prev
0,1375586,2015-01-28,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1050611,2015-01-28,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1050612,2015-01-28,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
# 前の月の製品情報が存在しない場合に備えて、0に代替します。
for prod in prods:
    prev = prod + '_prev'
    df_trn[prev].fillna(0, inplace=True)

In [75]:
# 訓練データから新規購買件数だけを抽出します。
X = []
Y = []
for i, prod in enumerate(prods):
    prev = prod + '_prev'
    prX = df_trn[(df_trn[prod] == 1) & (df_trn[prev] == 0)]
    prY = np.zeros(prX.shape[0], dtype=np.int8) + i
    X.append(prX)
    Y.append(prY)
XY = pd.concat(X)
Y = np.hstack(Y)
XY['y'] = Y

In [76]:
XY

,ncodpers,fecha_dato,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,...,ind_plan_fin_ult1_prev,ind_pres_fin_ult1_prev,ind_reca_fin_ult1_prev,ind_tjcr_fin_ult1_prev,ind_valo_fin_ult1_prev,ind_viv_fin_ult1_prev,ind_nomina_ult1_prev,ind_nom_pens_ult1_prev,ind_recibo_ult1_prev,y
314736,270730,2015-01-28,1,0,1,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
316383,268526,2015-01-28,1,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
317240,275486,2015-01-28,1,0,0,0,1,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
317686,278015,2015-01-28,1,0,1,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
322621,315724,2015-01-28,1,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11090455,1166385,2016-05-28,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23
11090468,1166355,2016-05-28,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23
11090667,1166343,2016-05-28,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23
11090696,1166232,2016-05-28,0,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,23


In [82]:
# XY[feats].values
XY['y'].values

array([ 0,  0,  0, ..., 23, 23, 23], dtype=int8)